In [28]:
import pandas as pd
import numpy as np
import os

output_path = "data"
os.makedirs(output_path, exist_ok=True)

In [29]:
df = pd.read_csv(r"DataCo Supply Chain Dataset.csv", encoding='latin-1') 
df

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.750000,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.750000,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.750000,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.750000,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.750000,0,1/15/2018 11:24,Standard Class
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,CASH,4,4,40.000000,399.980011,Shipping on time,0,45,Fishing,Brooklyn,...,NaN,1004,45,NaN,http://images.acmesports.sports/Field+%26+Stre...,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,0,1/20/2016 3:40,Standard Class
180515,DEBIT,3,2,-613.770019,395.980011,Late delivery,1,45,Fishing,Bakersfield,...,NaN,1004,45,NaN,http://images.acmesports.sports/Field+%26+Stre...,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,0,1/19/2016 1:34,Second Class
180516,TRANSFER,5,4,141.110001,391.980011,Late delivery,1,45,Fishing,Bristol,...,NaN,1004,45,NaN,http://images.acmesports.sports/Field+%26+Stre...,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,0,1/20/2016 21:00,Standard Class
180517,PAYMENT,3,4,186.229996,387.980011,Advance shipping,0,45,Fishing,Caguas,...,NaN,1004,45,NaN,http://images.acmesports.sports/Field+%26+Stre...,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,0,1/18/2016 20:18,Standard Class


In [30]:
# Assume df is already loaded
data = df.copy()

# Keep rows with valid business keys only
critical_cols = [
    'Order Id', 'Customer Id',
    'Product Name', 'Category Name',
    'Department Name'
]

data = (
    data
    .dropna(subset=critical_cols)
    .drop_duplicates()
    .reset_index(drop=True)
)

print("Clean rows:", len(data))


Clean rows: 180519


In [31]:
departments = (
    data[['Department Name']]
    .drop_duplicates()
    .reset_index(drop=True)
)

departments['department_id'] = departments.index + 1

departments = departments[['department_id', 'Department Name']]


In [32]:
categories = (
    data[['Category Name', 'Department Name']]
    .drop_duplicates()
    .reset_index(drop=True)
)

categories = categories.merge(
    departments,
    on='Department Name',
    how='left'
)

categories['category_id'] = categories.index + 1

categories = categories[['category_id', 'Category Name', 'department_id']]


In [33]:
customers = (
    data[
        [
            'Customer Id', 'Customer Fname', 'Customer Lname',
            'Customer Email', 'Customer Segment',
            'Customer City', 'Customer State',
            'Customer Country', 'Customer Zipcode', 'Customer Street'
        ]
    ]
    .drop_duplicates(subset=['Customer Id'])
    .reset_index(drop=True)
)

customers['customer_id'] = customers.index + 1

customer_key_map = customers[['Customer Id', 'customer_id']]

customers = customers.drop(columns=['Customer Id'])


In [34]:
products = (
    data[
        [
            'Product Name', 'Product Description',
            'Product Price', 'Product Status',
            'Category Name'
        ]
    ]
    .drop_duplicates(subset=['Product Name'])
    .reset_index(drop=True)
)

products = products.merge(
    categories[['Category Name', 'category_id']],
    on='Category Name',
    how='left'
)

products['product_id'] = products.index + 1

products = products[
    [
        'product_id', 'Product Name', 'Product Description',
        'Product Price', 'Product Status', 'category_id'
    ]
]


In [35]:
orders = (
    data[
        [
            'Order Id', 'order date (DateOrders)',
            'shipping date (DateOrders)', 'Order Status',
            'Shipping Mode', 'Order City', 'Order State',
            'Order Country', 'Order Zipcode', 'Order Region',
            'Late_delivery_risk'
        ]
    ]
    .drop_duplicates(subset=['Order Id'])
    .reset_index(drop=True)
)

orders = orders.merge(
    data[['Order Id', 'Customer Id']],
    on='Order Id',
    how='left'
)

orders = orders.merge(
    customer_key_map,
    on='Customer Id',
    how='left'
)

orders['order_id'] = orders.index + 1

order_key_map = orders[['Order Id', 'order_id']]

orders = orders.drop(columns=['Order Id', 'Customer Id'])


In [36]:
order_items = (
    data[
        [
            'Order Item Id', 'Order Id', 'Product Name',
            'Order Item Quantity', 'Order Item Product Price',
            'Order Item Discount', 'Order Item Discount Rate',
            'Order Item Total', 'Order Profit Per Order'
        ]
    ]
    .drop_duplicates()
    .reset_index(drop=True)
)

order_items = order_items.merge(
    order_key_map,
    on='Order Id',
    how='left'
)

order_items = order_items.merge(
    products[['Product Name', 'product_id']],
    on='Product Name',
    how='left'
)

order_items['order_item_id'] = order_items.index + 1

order_items = order_items[
    [
        'order_item_id', 'order_id', 'product_id',
        'Order Item Quantity', 'Order Item Product Price',
        'Order Item Discount', 'Order Item Discount Rate',
        'Order Item Total', 'Order Profit Per Order'
    ]
]


In [37]:
datasets = {
    "departments": departments,
    "categories": categories,
    "customers": customers,
    "products": products,
    "orders": orders,
    "order_items": order_items
}

for name, df_ in datasets.items():
    print(f"{name.upper()}")
    print("Rows:", len(df_))
    print("Nulls:", df_.isnull().sum().sum())
    print("Duplicates:", df_.duplicated().sum())
    print("-" * 40)


DEPARTMENTS
Rows: 11
Nulls: 0
Duplicates: 0
----------------------------------------
CATEGORIES
Rows: 51
Nulls: 0
Duplicates: 0
----------------------------------------
CUSTOMERS
Rows: 20652
Nulls: 11
Duplicates: 0
----------------------------------------
PRODUCTS
Rows: 129
Nulls: 129
Duplicates: 0
----------------------------------------
ORDERS
Rows: 180519
Nulls: 155679
Duplicates: 0
----------------------------------------
ORDER_ITEMS
Rows: 650954
Nulls: 0
Duplicates: 0
----------------------------------------


In [40]:
# Save each dataframe

orders.to_csv(f"{output_path}/orders.csv", index=False)


print("✅ All datasets saved successfully as CSV files")


✅ All datasets saved successfully as CSV files
